# Recommender System based on Review Book in E-commerce Using word2vec Algorithm


1. Yolanda R.M Manurung (12S17017)
2. Stella Sitinjak      (12S17027)
3. Ekis Naomi Lasma     (12S17059)


### Dataset
Data yang digunakan pada proyek ini berasal dari kaggle (https://www.kaggle.com/gapple/goodreads-book-reviews-20k)

## 1. Load Data

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#import library yang akan digunakan
import pandas as pd
import re
import gensim
import string
from gensim.models import Word2Vec
import numpy as np
from scipy import linalg
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet

In [3]:
#load dataset Goodreads buku dan menampilkan 5 data teratas
data = pd.read_csv("./all_data.csv")
print ('Jumlah data =', len(data))
data.head()

Jumlah data = 21559


,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\r\nThat’s the number of times that the wo...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\r\n""Rhys stepped off the stairs and...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


## 2. Data Preparation

Tahap awal sebelum preprocessing yaitu melakukan preparation untuk pengecekan data noise dan dirty. Tahap yang dilakukan adalah pengecekan null value dan duplicate value

### Drop Null Value

In [4]:
#pengecekan nilai null pada data
data.isnull().sum()

book_title                0
Book_series           12779
book_series_url       12779
book_image                2
book_image_url            2
book_rating               0
book_author               0
author_url                0
genre                   121
reviewer_name            25
reviewer_url             25
reviewer_image           25
reviewer_image_url       25
review                  742
ID                        0
dtype: int64

In [5]:
#penghapusan null value dengan menggunakan fillna
data['Book_series'] = data['Book_series'].fillna(data['Book_series'].mode().iloc[0])
data['book_series_url'] = data['book_series_url'].fillna(data['book_series_url'].mode().iloc[0])
data['book_image'] = data['book_image'].fillna(data['book_image'].mode().iloc[0])
data['book_image_url'] = data['book_image_url'].fillna(data['book_image_url'].mode().iloc[0])
data['genre'] = data['genre'].fillna(data['genre'].mode().iloc[0])
data['reviewer_name'] = data['reviewer_name'].fillna(data['reviewer_name'].mode().iloc[0])
data['reviewer_url'] = data['reviewer_url'].fillna(data['reviewer_url'].mode().iloc[0])
data['reviewer_image'] = data['reviewer_image'].fillna(data['reviewer_image'].mode().iloc[0])
data['reviewer_image_url'] = data['reviewer_image_url'].fillna(data['reviewer_image_url'].mode().iloc[0])
data['review'] = data['review'].fillna(data['review'].mode().iloc[0])

In [6]:
#pengecekan data setelah drop null value
data.isnull().sum()

book_title            0
Book_series           0
book_series_url       0
book_image            0
book_image_url        0
book_rating           0
book_author           0
author_url            0
genre                 0
reviewer_name         0
reviewer_url          0
reviewer_image        0
reviewer_image_url    0
review                0
ID                    0
dtype: int64

### Drop Duplicate Value

In [7]:
#pengecekan duplicate value 
data.duplicated().sum()

0

In [8]:
#menampilkan 5 data teratas yang sudah di cek duplicate value
data['book_title'].drop_duplicates(keep=False,inplace=True)
data.head()

,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\r\nThat’s the number of times that the wo...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\r\n""Rhys stepped off the stairs and...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


## 3. Data Preprocessing

### Case Folding

Proses untuk melakukan konversi seluruh karakter ke dalam huruf kecil

In [9]:
#mengubah huruf kapital menjadi huruf kecil yang ditampung pada parameter "lower"
review = []
for i in range(0, 21559):
    review.append(data["review"][i].casefold())
    
data['lower'] = review
data['lower'].head()

0    202.\r\nthat’s the number of times that the wo...
1    i need a black silk robe and a balcony to wear...
2    this book totally wrecked me. in the best poss...
3    3.75 stars\r\n"rhys stepped off the stairs and...
4    i wish so much that i could tell you i loved t...
Name: lower, dtype: object

### Remove Punctuation

In [10]:
#menghilangkan tanda baca dan ditampung oleh atribut baru yaitu "no_punc"
review = []
for i in range(0, 21559):
    review.append(repr(data["lower"][i].replace(",", "").replace("\n", "")
                                       .replace("\r", "").replace("---", "")
                                       .replace('|','').replace("!","")
                                       .replace("//", "").replace("?","")
                                       .replace("    ", " ").replace("  "," ")))

In [11]:
reviews = []
for i in review:
    review = re.sub(r'\ by.*? on', "", i)
    review = re.sub(r"\d\.\d", "", review)
    review = re.sub(r"\d+\ [a-z]+\.*\ \d+", "", review)
    review = review.replace(".", " ")
    review = re.sub(r'\  *', " ", review)
    reviews.append(review)
    
data["no_punc"] = reviews
data[["book_title", "book_rating", "genre", "no_punc"]].head()

,book_title,book_rating,genre,no_punc
0,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","""202 that’s the number of times that the word ..."
1,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",'i need a black silk robe and a balcony to wea...
2,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",'this book totally wrecked me in the best poss...
3,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","'5 stars""rhys stepped off the stairs and took ..."
4,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",'i wish so much that i could tell you i loved ...


Data yang sudah dilakukan preprocessing akan disimpan menjadi data baru dalam file DataBuku.csv

In [12]:
new = data[["ID", "book_title", "book_rating", "genre", "no_punc"]]
new.to_csv("./DataBuku.csv")

In [13]:
dataset = pd.read_csv("./Databuku.csv", delimiter=",")
dataset.head()

,Unnamed: 0,ID,book_title,book_rating,genre,no_punc
0,0,1,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","""202 that’s the number of times that the word ..."
1,1,2,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",'i need a black silk robe and a balcony to wea...
2,2,3,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",'this book totally wrecked me in the best poss...
3,3,4,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","'5 stars""rhys stepped off the stairs and took ..."
4,4,5,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",'i wish so much that i could tell you i loved ...


### Tokenization

In [14]:
#melakukan tokenisasi pada atribut "review" yang ditampung dalam parameter "tokenized"
from nltk.tokenize import SpaceTokenizer
sTokenizer = SpaceTokenizer()
review_tokens = []
for i in dataset['no_punc']:
    review_tokens.append(sTokenizer.tokenize(i))
    
dataset['tokenized'] = review_tokens
dataset['tokenized'].head()

0    ["202, that’s, the, number, of, times, that, t...
1    ['i, need, a, black, silk, robe, and, a, balco...
2    ['this, book, totally, wrecked, me, in, the, b...
3    ['5, stars"rhys, stepped, off, the, stairs, an...
4    ['i, wish, so, much, that, i, could, tell, you...
Name: tokenized, dtype: object

In [15]:
vocab = []
for i in dataset['tokenized']:
    vocab.append(set(i))
    
dataset['review_vocabs'] = vocab
dataset[["ID", "book_title", "book_rating", "genre", "review_vocabs"]].head()

,ID,book_title,book_rating,genre,review_vocabs
0,1,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","{catch, bored, saintly, clear, fifteen, body, ..."
1,2,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","{flaws, resurrected, making, hooo, underdevelo..."
2,3,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","{acomaf, past, accountable, clear, making, he,..."
3,4,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","{rated, tortured, acomaf, forgiving, bitched, ..."
4,5,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","{dude, series, four, i\'d, unsexy, liked, touc..."


## 4. Word Embedding with Word2Vec

In [16]:
word2vec_sg = Word2Vec(sentences=dataset['review_vocabs'],
                       size=100, window=5, min_count=1,
                       workers=4, sg=1, iter=10)

In [17]:
word2vec_sg.wv.similarity(w1="great",w2="good")

0.16326395

In [18]:
w1 = ["great"]
word2vec_sg.wv.most_similar (positive=w1,topn=6)

[('covers', 0.8304151296615601),
 ('female', 0.8221045732498169),
 ('myself', 0.810004472732544),
 ('returns', 0.8085720539093018),
 ('crafted', 0.8051140308380127),
 ('without', 0.802668035030365)]

In [19]:
w1 = ["good",'fun','great']
w2 = ['bad']
word2vec_sg.wv.most_similar (positive=w1,negative=w2,topn=10)

[('traveler', 0.8046613931655884),
 ('dirt', 0.8032265901565552),
 ('gaps', 0.8004559278488159),
 ('ite', 0.7993450164794922),
 ('excitement', 0.7959198951721191),
 ('newest', 0.7954776287078857),
 ('"holy', 0.7953488826751709),
 ('cindy', 0.7942132949829102),
 ('tag', 0.7898304462432861),
 ("white's", 0.7896919250488281)]

In [20]:
word2vec_sg.wv.similarity(w1="good",w2="good")

1.0

In [21]:
word2vec_sg.save("word2vec_model")
w2v = Word2Vec.load("word2vec_model")

In [22]:
book_data = []
for key,book in enumerate(dataset['review_vocabs']):
    if len(book) == 0:
        print(book, key)
    feature_vector = np.zeros((100,), dtype="float32")
    nwords = len(book)
    for token in book:
        feature_vector = np.add(feature_vector, w2v[token])
    
    feature_vector = np.divide(feature_vector, nwords)
    book_data.append({'book': dataset['book_title'][key],
                      'tags':book, 'vector':feature_vector})

In [23]:
book_vector_df = pd.DataFrame(book_data,columns=['book', 
                                                 'tags', 
                                                 'vector'])
book_vector_df[:10]

,book,tags,vector
0,A Court of Wings and Ruin,"{catch, bored, saintly, clear, fifteen, body, ...","[-0.006830589, 0.021674452, 0.27633604, -0.020..."
1,A Court of Wings and Ruin,"{flaws, resurrected, making, hooo, underdevelo...","[-0.012094354, 0.004298231, 0.2486076, -0.0332..."
2,A Court of Wings and Ruin,"{acomaf, past, accountable, clear, making, he,...","[-0.020606145, 0.020622712, 0.2896416, -0.0146..."
3,A Court of Wings and Ruin,"{rated, tortured, acomaf, forgiving, bitched, ...","[-0.017666489, 0.025592921, 0.26792833, -0.028..."
4,A Court of Wings and Ruin,"{dude, series, four, i\'d, unsexy, liked, touc...","[-0.03757106, -0.026826343, 0.28536475, -0.003..."
5,A Court of Wings and Ruin,"{series, swear, spring, thrilling, trilogy, ac...","[-0.042838484, -0.06484379, 0.2860937, -0.0088..."
6,A Court of Wings and Ruin,"{forgiven, series, any, take, he, amaze, them,...","[-0.079375714, -0.05378054, 0.2849432, 0.00378..."
7,A Court of Wings and Ruin,"{wrapped, hands, them, lack, this, thinking, a...","[-0.0921351, -0.0795774, 0.248911, -0.01134524..."
8,A Court of Wings and Ruin,"{""in, say, sobbing, reason, at, review, days, ...","[-0.11087561, -0.04326821, 0.21955167, -0.0693..."
9,A Court of Wings and Ruin,"{love, series, completely, review, hope, thril...","[-0.095561184, 0.057735626, 0.19862364, -0.017..."


In [24]:
book_vector_df.shape

(21559, 3)

## 5. Book Recommendation

In [25]:
words_in_vocab= set(word2vec_sg.wv.index2word)

In [26]:
def get_cosine_similarity(row, search_vector):
    try:
        cosine_similarity = np.dot(row['vector'], search_vector)/linalg.norm(row['vector'])/linalg.norm(search_vector)
        return cosine_similarity
    except Exception as e:
        raise e

In [27]:
# Mencari buku dengan judul "A Crown of Wishes"
book = input('Buku yang dicari: ')

Buku yang dicari: A Crown of Wishes


In [28]:
index_similar_book = 21559
for i, j in enumerate(dataset['book_title']):
    if j == book:
        index_similar_book = i

if index_similar_book != 21559:
    search_query = dataset['review_vocabs'][index_similar_book]

In [29]:
search_vector = np.zeros((100,), dtype="float32")
for tag in search_query:
    if tag in words_in_vocab:
        search_vector = np.add(search_vector, w2v[tag])
        
book_vector_df['cosine_similarity'] = book_vector_df.apply(get_cosine_similarity,
                                                           axis=1, args=(search_vector,))

In [30]:
df = book_vector_df.sort_values(by='cosine_similarity', ascending=False)[:10]

In [31]:
if index_similar_book == 21559:
    print("Buku {0} tidak ditemukan".format(book))
else:
    print("Maka buku yang direkomendasikan berdasarkan buku {0} adalah :".format(book))
    display(df[['book', 'cosine_similarity']])

Maka buku yang direkomendasikan berdasarkan buku A Crown of Wishes adalah :


,book,cosine_similarity
5740,A Crown of Wishes,1.000000
15384,Walking on the Sea of Clouds,0.996974
571,The Seven Husbands of Evelyn Hugo,0.996956
5969,The Hungry Brain: Outsmarting the Instincts Th...,0.996908
6330,Trickery,0.996899
7888,Ghosted,0.996693
1800,Without Merit,0.996607
3553,A Piece of the World,0.996570
3870,When It's Real,0.996567
3814,My Absolute Darling,0.996552
